In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
file_yelp = pd.read_csv('yelp_labelled.txt', names=['sentence', 'label'], sep='\t')
sentences = file_yelp['sentence'].values
y = file_yelp['label'].values
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.25, random_state=1000)

vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)
X_train = vectorizer.transform(sentences_train)
X_test = vectorizer.transform(sentences_test)

In [4]:
dim = X_train.shape[1]
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(10, input_dim = dim, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')  
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=100, verbose=False, validation_data=(X_test, y_test), batch_size=10)

In [6]:
print(sentences_test[100])
model.predict(X_test[100])

You get incredibly fresh fish, prepared with care.


array([[0.9937801]], dtype=float32)

In [19]:
new_sentences = np.array(['Unfortunately my one and only experience with Symmetry was hugely disappointing. I hope they take this review as a training opportunity.', 
                          'you always have enough room to feel comfortable doing all poses. Im a monthly member and will continue to be one as long as I can',
                          'The space is beautiful and air feels fresh. I will say the front desk staff is not particularly friendly.'], 
                         dtype=object)
X_new_test = vectorizer.transform(new_sentences)
X_new_test

<3x1714 sparse matrix of type '<class 'numpy.int64'>'
	with 46 stored elements in Compressed Sparse Row format>

In [20]:
prediction = model.predict(X_new_test[2])
if prediction[0][0] > 0.6:
  print(":)")
else:
  print(":(")


:)


In [21]:
keras_file='cf.h5'
tf.keras.models.save_model(model,keras_file)

convert_file = 'degree.tflite'
mdl = tf.keras.models.load_model(keras_file)
converter = tf.lite.TFLiteConverter.from_keras_model(mdl)
tflite_model = converter.convert()
open(convert_file, 'wb').write(tflite_model)

69700